# python小技巧：一个简单好用的用于抓取异常的decorator

写了一个提升工作效率的python的代码装饰器。

事情的背景是这样的：准备为当前流水线上提交一段功能代码，由于流水线为了保证不被代码可能的运行时错误而导致流水线失败，所以流水线本身采取了抓取异常日志但不抛出的设计（我从实际调试的现象来估计是这样的）。但是，由于每次都要线上验证功能正确性，如果没有一个有效的日志的话，就很麻烦。而且问题定位的时候很痛苦，因为流水线构建需要时间和重复的劳动，很耗费青春很痛苦。

而且，由于日志和报错栈被流水线的异常机制屏蔽掉了，每次流水线报错，只报出实际出错的位置，而不能够有效的显示这个出错的位置是在哪里被调用的，以及嵌套调用的关系。如果用打断点的方式一点点定位问题，这个时间开销太恐怖了。

于是写了一个简单的装饰器，但是好用。首先这个函数可以简单的表达函数的一个执行状态：执行中，成功或者失败（只要大家统一用相同的装饰器，那么日志上就很统一）。此外，由于装饰器的效果可以作用于函数嵌套，所以如果函数内部调用的子方法抛出了异常，那这个调用关系也能从日志中追踪，很高效。

最近的工作感悟：人之所以是人类的一个很大的里程碑是学会制造工具并且复用工具。如果可以的话，将流程自动化。重复劳动是一种罪孽。工具化，制度化，流程化，最后自动化。

额，闲话不多说了。下面是装饰器实现：

In [7]:
from loguru import logger
from func_timeout import func_set_timeout, func_timeout, FunctionTimedOut

def try_and_catch_exception(func):
    def wrapper(*args, **kwargs):
        logger.info(f"{func.__name__}: start running")
        result = None
        try:
            result = func(*args, **kwargs)
        except FunctionTimedOut as e:
            err_info = f"{func.__name__}: run timeout! info: {{ {e} }} "
            logger.error(err_info)
            raise RuntimeError(err_info)
        except Exception as e:
            err_info = f"{func.__name__}: error occur! info: {{ {e} }} "
            logger.error(err_info)
            raise RuntimeError(err_info)
        else:
            logger.info(f"{func.__name__}: succeed")
            return result
    return wrapper

下面是如何使用这个装饰器：

In [8]:
@try_and_catch_exception
def workwork(input_a, input_b):
    result = input_a * input_b
    return result

print(f"result: {workwork(2,4)}")

2023-02-16 01:04:19.594 | INFO     | __main__:wrapper:6 - workwork: start running
2023-02-16 01:04:19.596 | INFO     | __main__:wrapper:19 - workwork: succeed


result: 8


In [9]:
@try_and_catch_exception
def bad_workwork(input_a, input_b):
    result = input_a * input_b
    wrong = 1 / 0
    return result

try:
    print(f"result: {bad_workwork(2,4)}")
except Exception as e:
    print(e)

2023-02-16 01:04:19.691 | INFO     | __main__:wrapper:6 - bad_workwork: start running
2023-02-16 01:04:19.692 | ERROR    | __main__:wrapper:16 - bad_workwork: error occur! info: { division by zero } 


bad_workwork: error occur! info: { division by zero } 


当存在多个函数嵌套的时候，我们可以有一个大致的定位方向（因为报错栈有时候打印的信息有点太多了，看起来有点头大）。比方说：

In [10]:
@try_and_catch_exception
def a_func_use_bad_workwork():
    print("call bad_workwork...")
    print(f"result: {bad_workwork(2,4)}")

try:
    a_func_use_bad_workwork()
except Exception as e:
    logger.error(e)

2023-02-16 01:04:19.841 | INFO     | __main__:wrapper:6 - a_func_use_bad_workwork: start running
2023-02-16 01:04:19.842 | INFO     | __main__:wrapper:6 - bad_workwork: start running
2023-02-16 01:04:19.843 | ERROR    | __main__:wrapper:16 - bad_workwork: error occur! info: { division by zero } 
2023-02-16 01:04:19.844 | ERROR    | __main__:wrapper:16 - a_func_use_bad_workwork: error occur! info: { bad_workwork: error occur! info: { division by zero }  } 
2023-02-16 01:04:19.845 | ERROR    | __main__:<module>:9 - a_func_use_bad_workwork: error occur! info: { bad_workwork: error occur! info: { division by zero }  } 


call bad_workwork...


我们可以看到日志中清楚的记录了有用的报错信息。在代码比较多的时候，有这样的一句话能够比较清楚的表达出报错的具体位置，以及出错位置和其他函数之间的调用关系，这对问题定位有很大帮助。

下面再展示一个代码超时的功能，也非常的好用：

In [11]:
import time

@try_and_catch_exception
@func_set_timeout(5)    # timeout limit 5secs
def work_10secs():
    print("call work_10secs...")
    time.sleep(10)
    return True

try:
    print(work_10secs())
except Exception as e:
    logger.error(e)

2023-02-16 01:04:19.958 | INFO     | __main__:wrapper:6 - work_10secs: start running


call work_10secs...


2023-02-16 01:04:25.074 | ERROR    | __main__:wrapper:12 - work_10secs: run timeout! info: { Function work_10secs (args=()) (kwargs={}) timed out after 5.000000 seconds.
 } 
2023-02-16 01:04:25.075 | ERROR    | __main__:<module>:13 - work_10secs: run timeout! info: { Function work_10secs (args=()) (kwargs={}) timed out after 5.000000 seconds.
 } 
